In [4]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [2]:

# Load the dataset
data = pd.read_csv('./labeled_data.csv')

,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet
0,0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...
...,...,...,...,...,...,...,...
24778,25291,3,0,2,1,1,you's a muthaf***in lie &#8220;@LifeAsKing: @2...
24779,25292,3,0,1,2,2,"you've gone and broke the wrong heart baby, an..."
24780,25294,3,0,3,0,1,young buck wanna eat!!.. dat nigguh like I ain...
24781,25295,6,0,6,0,1,youu got wild bitches tellin you lies


In [7]:
# Extract the text and label columns
text = data['tweet']
labels = data['class']

# Tokenize
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text)
sequences = tokenizer.texts_to_sequences(text)
vocab_size = len(tokenizer.word_index) + 1

# Padding
max_length = max(len(seq) for seq in sequences)
padded_sequences = pad_sequences(sequences, maxlen=max_length)
max_sequence_length = max(len(seq) for seq in sequences)

# this for num of labels (harusnya ada 3)
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)
num_classes = len(label_encoder.classes_)
one_hot_labels = to_categorical(encoded_labels)

X_train, X_test, y_train, y_test = train_test_split(padded_sequences, one_hot_labels, test_size=0.2, random_state=42)

# Define the LSTM model
model = Sequential()
model.add(Embedding(vocab_size, 100, input_length=max_sequence_length))
model.add(LSTM(64, dropout=0.2, recurrent_dropout=0.2))
model.add(Dropout(0.5))
model.add(Dense(128, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=32)

Epoch 1/10
620/620 [==============================] - 39s 60ms/step - loss: 0.2725 - accuracy: 0.8484 - val_loss: 0.2039 - val_accuracy: 0.8874
Epoch 2/10
620/620 [==============================] - 39s 63ms/step - loss: 0.1523 - accuracy: 0.9177 - val_loss: 0.2012 - val_accuracy: 0.8858
Epoch 3/10
620/620 [==============================] - 39s 63ms/step - loss: 0.0943 - accuracy: 0.9493 - val_loss: 0.2698 - val_accuracy: 0.8737
Epoch 4/10
620/620 [==============================] - 37s 60ms/step - loss: 0.0549 - accuracy: 0.9715 - val_loss: 0.3190 - val_accuracy: 0.8662
Epoch 5/10
620/620 [==============================] - 37s 60ms/step - loss: 0.0352 - accuracy: 0.9826 - val_loss: 0.3754 - val_accuracy: 0.8761
Epoch 6/10
620/620 [==============================] - 39s 62ms/step - loss: 0.0254 - accuracy: 0.9879 - val_loss: 0.3893 - val_accuracy: 0.8584
Epoch 7/10
620/620 [==============================] - 38s 62ms/step - loss: 0.0175 - accuracy: 0.9916 - val_loss: 0.4631 - val_accuracy:

In [6]:
import nltk
from nltk.corpus import stopwords
import re

### Preprocessing
it consist of:
- removing unique char (emoji, flags, non-alphabet char)
- url links
- stopwords
- changing @xxx into user
- null tweets

In [10]:
stop_words = set(stopwords.words('english'))

def remove_emojis(raw_text):
    emoji = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
                      "]+", re.UNICODE)
    
    text = re.sub(emoji, '', raw_text)
    return text

def remove_stopwords(raw_text):
    tokenize = nltk.word_tokenize(raw_text)
    text = [word for word in tokenize if not word.lower() in stop_words]
    text = " ".join(text)

    return text
# Preprocessing
def remove_url(raw_text):
    url_regex = r"(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'\".,<>?«»“”‘’]))"
    text = re.sub(url_regex, '', raw_text)

    return text

def change_user(raw_text):
    regex = r"@([^ ]+)"
    text = re.sub(regex, "user", raw_text)
    return text
        

In [8]:
test_list = [
    'got ya bitch tip toeing on my hardwood floors "" &#128514; http://t.co/cOU2WQ5L4q"',
    "pussy is a powerful drug @juanwmv "" &#128517; #HappyHumpDay http://t.co/R8jsymiB5b",
    "...Son of a bitch took my Tic Tacs.",
    '"@2015seniorprobs: I probably wouldn&#8217;t mind school as much if we didn&#8217;t have to deal with bitch ass teachers"". Retweet',
    '"""..All I wanna do is get money and fuck model bitches!"" - Russell Simmons"',
    "@AutoWorId: Hennessey Venom GT &#128584; http://t.co/i8eGMnKaJ9"" that's one sexy bitch"
]

for x in test_list:
    print(x)

got ya bitch tip toeing on my hardwood floors "" &#128514; http://t.co/cOU2WQ5L4q"
pussy is a powerful drug @juanwmv  &#128517; #HappyHumpDay http://t.co/R8jsymiB5b
...Son of a bitch took my Tic Tacs.
"@2015seniorprobs: I probably wouldn&#8217;t mind school as much if we didn&#8217;t have to deal with bitch ass teachers"". Retweet
"""..All I wanna do is get money and fuck model bitches!"" - Russell Simmons"
@AutoWorId: Hennessey Venom GT &#128584; http://t.co/i8eGMnKaJ9 that's one sexy bitch


In [11]:
def remove_noise(datas):
    clean = []
    # change the @xxx into "user"
    clean = [change_user(text) for text in datas]
    # remove emojis (specifically unicode emojis)
    clean = [remove_emojis(text) for text in clean]
    # remove urls
    clean = [remove_url(text) for text in clean]
    # remove stopwords
    clean = [remove_stopwords(text) for text in clean]
    return clean

test_list = remove_noise(test_list)

In [13]:
for x in test_list:
    print(x)

got ya bitch tip toeing hardwood floors `` '' & # 128514 ; ``
pussy powerful drug user & # 128517 ; # HappyHumpDay
... Son bitch took Tic Tacs .
`` user probably & # 8217 ; mind school much & # 8217 ; deal bitch ass teachers '' '' . Retweet
`` `` '' .. wan na get money fuck model bitches ! '' '' - Russell Simmons ''
user Hennessey Venom GT & # 128584 ; 's one sexy bitch


In [ ]:
# Split into training and testing sets
train_text, test_text, train_labels, test_labels = train_test_split(text, labels, test_size=0.2, random_state=42)

# Tokenize the text
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_text)
vocab_size = len(tokenizer.word_index) + 1

# Convert text to sequences
train_sequences = tokenizer.texts_to_sequences(train_text)
test_sequences = tokenizer.texts_to_sequences(test_text)

# Pad sequences to ensure uniform length
max_sequence_length = max(len(seq) for seq in train_sequences)
train_sequences = pad_sequences(train_sequences, maxlen=max_sequence_length, padding='post')
test_sequences = pad_sequences(test_sequences, maxlen=max_sequence_length, padding='post')

# Define the LSTM model
model = Sequential()
model.add(Embedding(vocab_size, 100, input_length=max_sequence_length))
model.add(LSTM(64, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='softmax'))

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(train_sequences, train_labels, epochs=10, batch_size=32, validation_split=0.2)

# Evaluate the model
test_loss, test_acc = model.evaluate(test_sequences, test_labels)

# Make predictions
predictions = model.predict(test_sequences)